In [ ]:
import boto3
import time
import csv

In [ ]:
## Textract APIs used - "start_document_text_detection", "get_document_text_detection"
def InvokeTextDetectJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract')
    response = client.start_document_text_detection(
            DocumentLocation={
                      'S3Object': {
                                    'Bucket': s3BucketName,
                                    'Name': objectName
                                }
           })
    return response["JobId"]

def CheckJobComplete(jobId):
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    return status

def JobResults(jobId):
    pages = []
    client = boto3.client('textract')
    response = client.get_document_text_detection(JobId=jobId)
 
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

In [ ]:
# question number to string
def num_to_string(i):
    return str(i) + '.'

In [ ]:
# S3 Document Data
# Use the bucket with your added survey and the file name

#s3BucketName = "549projectbucket"
s3BucketName = "norfolk"
documentName = "31.pdf"

# Function invokes
jobId = InvokeTextDetectJob(s3BucketName, documentName)
print("Started job with id: {}".format(jobId))
print(documentName)
if(CheckJobComplete(jobId)):
    response = JobResults(jobId)

#Process each surveys
result = []
block = []
for resultPage in response:
    for item in resultPage["Blocks"]:
        if item["BlockType"] == "LINE":
                result.append(item["Text"])                
print(result)

In [ ]:
# Extracting data between questions
ques_num = 1
while not ques_num == 66:
    for i in range(len(result)):
        if num_to_string(ques_num) in result[i]:
            for j in range(i+1, len(result)):
                if num_to_string(ques_num + 1) in result[j]:    
                        ans = ""
                        for k in range(i+1,j):
                                ans += result[k] + ' '
                        block.append(ans)
                        ques_num += 1
                        break

                        
# Get the answer for the last question
for i in range(len(result)):
    if num_to_string(66) in result[i]:
        ans = ""
        for k in range(i+1, len(result)):
            ans += result[k] + ' '
        block.append(ans)
        break

        
# remove unwanted data

for i in range(len(block)):
    if "Yes No Prefer not to answer" in block[i]:
        block[i] = " "
    elif "Yes No I don't know" in block[i]:
        block[i] = " "
    elif "Straight (Heterosexual) Lesbian Gay Bisexual Asexual" in block[i]:
        block[i] = " "
    elif "Male Female Prefer to self-describe:" in block[i]:
        block[i] = " "
    elif "Black/African American Asian White Native American Native Hawaiian/Pacific Islander Multi-Race Middle Eastern/North African" in block[i]:
        block[i] = " "
    elif "Never About once per year A few times a month" in block[i]:
        block[i] = " "
    elif "How long did you spend in foster care?" in block[i]:
        block[i] = " "
    elif "Longer than one year" in block[i]:
        block[i] = " "
    elif "While in foster care" in block[i]:
        block[i] = " "
    elif "Once Once Once" in block[i]:
        block[i] = " "
    elif "Yes Yes Yes" in block[i]:
        block[i] = " "
    elif "One One One" in block[i]:
        block[i] = " "
         

    # remove unwanted data
new_block = block
new_block = [x.replace('around, or just the city and state where you grew up). ','') for x in new_block]
new_block = [x.replace(' 2 CHILDHOOD (all of these questions refer to your first 18 years of life) ','')  for x in new_block]
new_block = [x.replace('Parent Other family member Teacher/coach Foster Care Other:','') for x in new_block]
new_block = [x.replace('Yes If yes: what did that sport/club mean to you? No ','') for x in new_block]
new_block = [x.replace('Yes If yes: Before you were 18 did they ever separate or divorce? Yes If no please explain:','') for x in new_block]
new_block = [x.replace('Yes please explain:','') for x in new_block]
new_block = [x.replace('SPECIAL EDUCATION SERVICES (while in elementary school, middle school or high school)','') for x in new_block]
new_block = [x.replace('Yes 1. What services did you receive through your IEP? (Check all that apply). Emotional therapy Behavioral therapy Speech/Language therapy Physical/occupational therapy Other: 2. What was your disability?','') for x in new_block]
new_block = [x.replace('you needed? Yes - explain: No - explain: ','') for x in new_block]
new_block = [x.replace('Yes If yes: what services did you receive through your 504 plan? No','') for x in new_block]
new_block = [x.replace('Never Sometimes Often Very often ','') for x in new_block]
new_block = [x.replace(' 7 JUVENILE JUSTICE INVOLVEMENT ','') for x in new_block]
new_block = [x.replace('Yes - for a few days Yes - for a few weeks No ','') for x in new_block]
new_block = [x.replace('If so, what were they?','') for x in new_block]
new_block = [x.replace('Yes 1. If yes: what were you arrested for?','') for x in new_block]
new_block = [x.replace('How many times were you arrested under the age of 18?','') for x in new_block]
new_block = [x.replace('first involved?','') for x in new_block]
new_block = [x.replace('justice system? Mostly positive Mixed Mostly negative Please explain: ','') for x in new_block]
new_block = [x.replace('Yes If yes: what were you convicted of? No 9 ','') for x in new_block]
new_block = [x.replace('10 SCHOOL Please fill out the chart below by checking the box for each question that describes your experiences in Elementary, Middle and High School (please check one box in each school column) Elementary School Middle School High School ','') for x in new_block]
new_block = [x.replace('your first incarceration?','') for x in new_block]
new_block = [x.replace('In-school In-school In-school in-school or out-of- Out-of-school Out-of-school Out-of-school school? If yes, what were you suspended for? ','') for x in new_block]
new_block = [x.replace('reaction to the survey, and/or your ideas about how these systems could be improved below. ','') for x in new_block]
new_block = [x.replace(' 12 ','') for x in new_block]
new_block = [x.replace(' 11 ','') for x in new_block]
new_block = [x.replace('6 MENTAL HEALTH ','') for x in new_block]
new_block = [x.replace(' 1 ','') for x in new_block]
new_block = [x.replace('having trouble with reading or math? Yes If yes: What was your learning disability?','') for x in new_block]
new_block = [x.replace('depression, anxiety, PTSD, or a mood or psychotic disorder? Yes If yes: what condition(s)','') for x in new_block]
new_block = [x.replace('Straight (Heterosexual) Lesbian Gay Bisexual Prefer Asexual to self-describe: ',' ') for x in new_block]
new_block = [x.replace('Detained Sentenced If sentenced: how long is your sentence?','') for x in new_block]
new_block = [x.replace(' No 9 ','') for x in new_block]
new_block = [x.replace('Yes If yes: what services did you receive through your 504 plan?','') for x in new_block]    
new_block = [x.replace(' 5 ','') for x in new_block]
new_block = [x.replace(' 3 ','') for x in new_block]
new_block = [x.replace('Yes No','') for x in new_block]

    # add output of 1 survey to a list
new_block.insert(0,documentName)

In [ ]:
# Add new survey to exisiting csv
file_name = 'Framingham.csv'
file_name2 = 'Norfolk.csv'
with open(file_name2, 'a') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(new_block)
    csv_file.close()